# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [117]:
# import numpy and pandas
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import t
from statsmodels.stats.proportion import proportions_ztest

# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents

In [7]:
# Run this code:

salaries = pd.read_csv('../../Current_Employee_Names__Salaries__and_Position_Titles.csv')

Examine the `salaries` dataset using the `head` function below.

In [9]:
# Your code here:
salaries.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [10]:
# Your code here:
salaries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33183 entries, 0 to 33182
Data columns (total 8 columns):
Name                 33183 non-null object
Job Titles           33183 non-null object
Department           33183 non-null object
Full or Part-Time    33183 non-null object
Salary or Hourly     33183 non-null object
Typical Hours        8022 non-null float64
Annual Salary        25161 non-null float64
Hourly Rate          8022 non-null float64
dtypes: float64(3), object(5)
memory usage: 2.0+ MB


In [18]:
columns = salaries.columns

for column in columns:
    nan = salaries[column].isna().sum()
    print(f'The NaNs of the column {column} are {nan}')

The NaNs of the column Name are 0
The NaNs of the column Job Titles are 0
The NaNs of the column Department are 0
The NaNs of the column Full or Part-Time are 0
The NaNs of the column Salary or Hourly are 0
The NaNs of the column Typical Hours are 25161
The NaNs of the column Annual Salary are 8022
The NaNs of the column Hourly Rate are 25161


Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [22]:
# Your code here:
salaried = salaries['Salary or Hourly'].loc[salaries['Salary or Hourly'] == 'Salary'].count()
hourlied = salaries['Salary or Hourly'].loc[salaries['Salary or Hourly'] == 'Hourly'].count()

print(f'The Salary pays are {salaried}')
print(f'The Hourly pays are {hourlied}')


The Salary pays are 25161
The Hourly pays are 8022


What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [115]:
# Your code here:
departments = salaries[['Name','Department']].groupby('Department').agg('count').sort_values(by='Name', ascending=False).reset_index()
departments.columns = ['Department','Total Employees']
departments.head()

,Department,Total Employees
0,POLICE,13414
1,FIRE,4641
2,STREETS & SAN,2198
3,OEMC,2102
4,WATER MGMNT,1879


# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [51]:
hourly_rates = salaries['Hourly Rate'].loc[salaries['Salary or Hourly'] == 'Hourly']

In [72]:
# Your code here:
m = 30
alpha = 0.05
ttest = stats.ttest_1samp(hourly_rates, m)
print(f'T-test statistic={ttest[0]}, pvalue={ttest[1]}')
alpha = 0.05
if (ttest[1] < alpha):
    print(f'Reject null hypothesis, mean is different from {m}')
else:
    print('Accept null hypothesis')

T-test statistic=20.6198057854942, pvalue=4.3230240486229894e-92
Reject null hypothesis, mean is different from 30


We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

In [69]:
police_salaries = salaries['Annual Salary'].loc[(salaries['Salary or Hourly'] == 'Salary') & (salaries['Department'] == 'POLICE')]


In [73]:
# Your code here:
m = 86000
alpha = 0.05
ttest = stats.ttest_1samp(police_salaries, m)
print(f'T-test statistic={ttest[0]}, pvalue={ttest[1]}')
alpha = 0.05
if (ttest[0] > 0) & (ttest[1]/2 < alpha):
    print(f'Reject null hypothesis, mean is greater than {m}')
else:
    print('Accept null hypothesis')

T-test statistic=3.081997005712994, pvalue=0.0020603403550965137
Reject null hypothesis, mean is greater than 86000


Using the `crosstab` function, find the department that has the most hourly workers. 

In [77]:
# Your code here:
pd.crosstab(salaries['Department'], salaries['Salary or Hourly']).sort_values(by='Hourly', ascending=False).reset_index().head()


Salary or Hourly,Department,Hourly,Salary
0,STREETS & SAN,1862,336
1,WATER MGMNT,1513,366
2,OEMC,1273,829
3,AVIATION,1082,547
4,GENERAL SERVICES,765,215


The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [80]:
hourly_rates_ss = salaries['Hourly Rate'].loc[(salaries['Department'] == 'STREETS & SAN') & (salaries['Salary or Hourly'] == 'Hourly')]


In [82]:
# Your code here:
m = 35
alpha = 0.05
ttest = stats.ttest_1samp(hourly_rates_ss, m)
print(f'T-test statistic={ttest[0]}, pvalue={ttest[1]}')
alpha = 0.05
if (ttest[0] < 0) & (ttest[1]/2 < alpha):
    print(f'Reject null hypothesis, mean is greater than {m}')
else:
    print('Accept null hypothesis')


T-test statistic=-9.567447887848152, pvalue=3.3378530564707717e-21
Reject null hypothesis, mean is greater than 35


# Challenge 3 - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [118]:
hourly_rates = salaries['Hourly Rate'].loc[salaries['Salary or Hourly'] == 'Hourly']

In [128]:
# Your code here:
mean = hourly_rates.mean()
st = stats.sem(hourly_rates)
alpha = 0.95
df = hourly_rates.count()-1
t.interval(alpha, df, loc=mean, scale=st) 

(32.52345834488425, 33.05365708767623)

Now construct the 95% confidence interval for all salaried employeed in the police in the cell below.

In [130]:
police_salaries = salaries['Annual Salary'].loc[(salaries['Salary or Hourly'] == 'Salary') & (salaries['Department'] == 'POLICE')]

In [131]:
# Your code here:
mean = police_salaries.mean()
st = stats.sem(police_salaries)
alpha = 0.95
df = police_salaries.count()-1
t.interval(alpha, df, loc=mean, scale=st) 

(86177.05631531784, 86795.77269094894)

# Challenge 4 - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [109]:
# Your code here:
null_hyp = 0.25
alpha = 0.05
hourly_workers = salaries['Salary or Hourly'].loc[salaries['Salary or Hourly'] == 'Hourly'].count()
total_workers = salaries['Salary or Hourly'].count()

In [116]:
ztest = proportions_ztest(hourly_workers , total_workers, null_hyp)
print(f'P value is {round(ztest[1],4)}')
if (ztest[1]< alpha):
    print(f'Reject null hypothesis, mean is different from {null_hyp*100}%')
else:
    print('Accept null hypothesis')


P value is 0.0004
Reject null hypothesis, mean is different from 25.0%
